In [1]:
import pandas as pd
import requests
import itertools
from bs4 import BeautifulSoup
from requests.exceptions import ChunkedEncodingError

In [2]:

# Read the URLs from the text file
with open('all_recipes_url.txt', 'r') as file:
    all_recipes_url = file.readlines()

# Strip newline characters from each URL
all_recipes_url = [url.strip() for url in all_recipes_url]

len(all_recipes_url)

157

In [3]:
all_recipes_url

['https://www.halfbakedharvest.com/peach-thyme-smash/',
 'https://www.halfbakedharvest.com/frozen-watermelon-spritz/',
 'https://www.halfbakedharvest.com/the-50-most-popular-summer-recipes-of-2024/',
 'https://www.halfbakedharvest.com/coconut-lime-margaritas/',
 'https://www.halfbakedharvest.com/white-russian/',
 'https://www.halfbakedharvest.com/gingerbread-cocktail/',
 'https://www.halfbakedharvest.com/popular-holiday-appetizers-and-cocktails-mocktails/',
 'https://www.halfbakedharvest.com/christmas-punch/',
 'https://www.halfbakedharvest.com/naughty-nice-margarita/',
 'https://www.halfbakedharvest.com/mistletoe-kiss-cocktail/',
 'https://www.halfbakedharvest.com/sage-ginger-paloma/',
 'https://www.halfbakedharvest.com/cranberry-cider-margarita/',
 'https://www.halfbakedharvest.com/cinnamon-apple-bourbon-smash/',
 'https://www.halfbakedharvest.com/the-24-most-popular-thanksgiving-drinks/',
 'https://www.halfbakedharvest.com/bloody-mule/',
 'https://www.halfbakedharvest.com/hocus-pocu

# Extract Information of all recipes

In [4]:
failed_url = []

In [5]:
# Empty Dataframe
columns = [
    'title',
    'prep_time',
    'cook_time',
    'total_time',
    'ingredients',
    'ingredients with measurement',
    'instructions',
    'calories',
    'servings',
    'images'
]

In [6]:
final = pd.DataFrame(columns=columns)
final

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images


# Extract Function

In [7]:
def extract(soup):
    try:
        # TITLE
        h2_tag = soup.find('h2', class_='wprm-recipe-name')  # Replaced temp_soup with soup
        if h2_tag:
            title.append(h2_tag.text.strip())
        else:
            title.append('-')

        # PREP TIME
        prep_time_elem = soup.find(class_='wprm-recipe-prep-time-container')  # Replaced temp_soup with soup
        if prep_time_elem:
            time_dict = {}
            hours_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-hours')
            minutes_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value
            prep_time.append(time_dict) 
        else:
            prep_time.append('-')

        # COOK TIME
        cook_time_elem = soup.find(class_='wprm-recipe-cook-time-container')  # Replaced temp_soup with soup
        if cook_time_elem:
            time_dict = {}
            hours_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-hours')
            minutes_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value

            cook_time.append(time_dict) 
        else:
            cook_time.append('-')

        # TOTAL TIME
        total_time_elem = soup.find(class_='wprm-recipe-total-time-container')  # Replaced temp_soup with soup
        if total_time_elem:
            time_dict = {}
            hours_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-hours')
            minutes_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value
            total_time.append(time_dict) 
        else:
            total_time.append('-')

        # INGREDIENTS
        ingr_list = soup.find_all('li', class_='wprm-recipe-ingredient')  # Replaced temp_soup with soup
        if ingr_list:
            ingredient_dict = {}
            ingredients_recipe = []
            for ingr in ingr_list:
                ingr_amount_elem = ingr.find(class_='wprm-recipe-ingredient-amount')
                ingr_amount = ingr_amount_elem.text.strip() if ingr_amount_elem else None
                ingr_name_elem = ingr.find(class_='wprm-recipe-ingredient-name')
                ingr_name = ingr_name_elem.text.strip() if ingr_name_elem else None
                ingredient_dict[ingr_name] = ingr_amount if ingr_amount else "seasoning"
                ingredients_recipe.append(ingr_name)
            ingredients_measurment.append(ingredient_dict)
            ingredients.append(ingredients_recipe)
        else:
            ingredients_measurment.append('-')
            ingredients.append('-')

        # INSTRUCTIONS
        recipe_instr = soup.find(class_='wprm-recipe-instruction-group')  # Replaced temp_soup with soup
        if recipe_instr:
            recipe_steps = []
            steps = recipe_instr.find_all("span", attrs={"class": None})
            for step in steps:
                recipe_steps.append(step.text.strip())
            instructions.append(recipe_steps)
        else:
            instructions.append('-')

        # SERVINGS
        serving_div = soup.find('span', class_='wprm-recipe-servings')  # Replaced temp_soup with soup
        if serving_div:
            servings.append(serving_div.text.strip())
        else:
            servings.append('-')

        # CALORIES
        calorie_div = soup.find('span',class_='wprm-recipe-nutrition-with-unit')  # Replaced temp_soup with soup
        if calorie_div:
            calories.append(calorie_div.text.strip())
        else:
            calories.append('-')

        # IMAGES
        images.append('-')
        img_div = soup.find(class_='entry-content')
        img_tags = img_div.find_all('img')
        jpg_links = [img['src'] for img in img_tags if img['src'].endswith('.jpg')]
        if len(jpg_links) >= 2:
            second_jpg_link = jpg_links[1]
            images[-1] = second_jpg_link

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        failed_url.append(url)  # Append failed URL to list


### Recipes 0-24

In [8]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_1 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(0, 150):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_1['title'] = title
temp_recipes_1['prep_time'] = prep_time
temp_recipes_1['cook_time'] = cook_time
temp_recipes_1['total_time'] = total_time
temp_recipes_1['ingredients'] = ingredients
temp_recipes_1['ingredients with measurement'] = ingredients_measurment
temp_recipes_1['instructions'] = instructions
temp_recipes_1['calories'] = calories
temp_recipes_1['servings'] = servings
temp_recipes_1['images']= images



    

Proxies loaded: ['160.86.242.23:8080', '185.195.71.218:18080', '35.220.254.137:8080']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved 0 - https://www.halfbakedharvest.com/peach-thyme-smash/ with proxy 160.86.242.23:8080
Proxy 185.195.71.218:18080 failed for https://www.halfbakedharvest.com/frozen-watermelon-spritz/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /frozen-watermelon-spritz/ (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 503 Service Unavailable')))
Successfully retrieved 2 - https://www.halfbakedharvest.com/the-50-most-popular-summer-recipes-of-2024/ with proxy 35.220.254.137:8080


...

<IPython.core.display.Javascript object>

Successfully retrieved 145 - https://www.halfbakedharvest.com/harvest-apple-ginger-spritz/ with proxy 185.195.71.218:18080
Proxy 35.220.254.137:8080 failed for https://www.halfbakedharvest.com/pumpkin-patch-punch/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /pumpkin-patch-punch/ (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1227dea60>: Failed to establish a new connection: [Errno 61] Connection refused')))
Proxy 160.86.242.23:8080 failed for https://www.halfbakedharvest.com/the-black-widow-smash/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /the-black-widow-smash/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved 148 - https://www.halfbakedharvest.com/the-half-blood-prince-cocktail/ with proxy 185.195.71.218:18080
Proxy 35.220.254.137:8080 failed for https://www.halfbakedharvest.com/poison-apple-martini/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /poison-apple-martini/ (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1225379a0>: Failed to establish a new connection: [Errno 61] Connection refused')))


In [9]:
print(len(failed_url))

45


In [10]:
failed_url

['https://www.halfbakedharvest.com/frozen-watermelon-spritz/',
 'https://www.halfbakedharvest.com/hocus-pocus-old-fashioned/',
 'https://www.halfbakedharvest.com/pomegranate-punch/',
 'https://www.halfbakedharvest.com/the-20-most-popular-fall-drinks/',
 'https://www.halfbakedharvest.com/frosted-mistletoe-margarita/',
 'https://www.halfbakedharvest.com/spiced-cranberry-punch/',
 'https://www.halfbakedharvest.com/the-14-most-popular-thanksgiving-drinks/',
 'https://www.halfbakedharvest.com/voldemorts-dark-serum/',
 'https://www.halfbakedharvest.com/frostys-frosted-moscow-mule/',
 'https://www.halfbakedharvest.com/cranberry-thyme-spritz/',
 'https://www.halfbakedharvest.com/vampires-drip-cocktail/',
 'https://www.halfbakedharvest.com/the-deathly-hallows-cocktail/',
 'https://www.halfbakedharvest.com/fig-honey-thyme-prosecco-smash/',
 'https://www.halfbakedharvest.com/blood-orange-champagne-mule/',
 'https://www.halfbakedharvest.com/christmas-ale-sangria/',
 'https://www.halfbakedharvest.c

In [11]:
temp_recipes_1

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Peach Thyme Smash,{'minutes': '5'},{'minutes': '5'},{'minutes': '10'},"[fresh peach slices, fresh thyme, peach jam/pr...","{'fresh peach slices': '1/2', 'fresh thyme': '...",[1. Optional: heat a grill or grill pan to hig...,226 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
1,-,-,-,-,-,-,-,-,-,https://www.halfbakedharvest.com/wp-content/up...
2,Batch Spicy Coconut Lime Margarita,{'minutes': '15'},-,{'minutes': '15'},"[jalapeños or serrano chiles, fresh basil leav...","{'jalapeños or serrano chiles': '2', 'fresh ba...",[1. Place the jalapeños (chiles) over a gas fl...,372 kcal,4,https://www.halfbakedharvest.com/wp-content/up...
3,Uncle Eddy’s White Russian,{'minutes': '10'},{'minutes': '5'},{'minutes': '15'},"[honey, cinnamon, nutmeg, vodka, bourbon or ru...","{'honey': '1/4', 'cinnamon': '2', 'nutmeg': '1...","[1. To make the cinnamon syrup. In a pot, comb...",373 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
4,Naughty Gingerbread Cocktail (+mocktail).,{'minutes': '5'},-,{'minutes': '7'},"[bourbon or spiced rum, orange or clementine j...","{'bourbon or spiced rum': '1 1/2', 'orange or ...","[1. If desired, rim your glass. On a small pla...",191 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
...,...,...,...,...,...,...,...,...,...,...
100,Winter Citrus Tequila Smash,{'minutes': '10'},-,{'minutes': '10'},"[orange zest, blood orange, peel removed and q...","{'orange zest': '1', 'blood orange, peel remov...","[1. In a cocktail shaker or glass jar, combine...",96 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
101,Christmas Snowstorm Margarita,{'minutes': '10'},-,{'minutes': '10'},"[unsweetened coconut milk, honey or granulated...","{'unsweetened coconut milk': '1 1/3', 'honey o...",[1. Bring the coconut milk and sugar to a simm...,252 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
102,Gingerbread White Russian,{'minutes': '5'},{'minutes': '5'},{'minutes': '10'},"[honey, molasses, fresh ginger, sliced, cinnam...","{'honey': '1/4', 'molasses': '3', 'fresh ginge...","[1. In a medium pot, combine 1 1/3 cups water,...",291 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
103,Harvest Apple Spritz.,{'minutes': '10'},{'minutes': '10'},{'minutes': '20'},"[honey, fresh ginger,, fresh sage leaves, plus...","{'honey': '1/4', 'fresh ginger,': '1', 'fresh ...","[1. In a small pot, bring 1/2 cup water, the h...",359 kcal,8,https://www.halfbakedharvest.com/wp-content/up...


In [12]:
title

['Peach Thyme Smash',
 '-',
 'Batch Spicy Coconut Lime Margarita',
 'Uncle Eddy’s White Russian',
 'Naughty Gingerbread Cocktail (+mocktail).',
 '-',
 'WhoVille Christmas Punch',
 'Naughty Nice Margarita',
 'Mistletoe Kiss Cocktail',
 'Sage Ginger Paloma',
 'Fireside Cranberry Cider Margarita',
 'Cinnamon Apple Bourbon Smash',
 '-',
 'Bloody Mule',
 '-',
 'Strawberry Lemonade Aperol Margarita.',
 'Spicy Jalapeño Ginger Moscow Mule',
 'Old Saint Nick’s Chocolate Espresso Martini',
 'Whoville’s Spiced Up Christmas Margarita (with mocktail)',
 'Christmas Punch',
 'Christmas Vacation Dirty Chai Cocktail\xa0(with\xa0mocktail)',
 '“I Put a Spell on You” Poison Apple Martini',
 'Frozen Peach Rosé Slushy',
 'Firecracker Champagne Paloma',
 'Frosted Watermelon Rosemary Frosé',
 'Spicy Serrano Pineapple Margarita',
 'Frozen Blueberry Paloma',
 'Citrus Pomegranate Champagne Smash',
 '-',
 'Creamy Coconut White Russian',
 'The Sweet and Spicy Cheermeister Cocktail',
 'Pomegranate Champagne Mule',


In [13]:
final = pd.concat([final, temp_recipes_1], ignore_index=True)

In [14]:
final

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Peach Thyme Smash,{'minutes': '5'},{'minutes': '5'},{'minutes': '10'},"[fresh peach slices, fresh thyme, peach jam/pr...","{'fresh peach slices': '1/2', 'fresh thyme': '...",[1. Optional: heat a grill or grill pan to hig...,226 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
1,-,-,-,-,-,-,-,-,-,https://www.halfbakedharvest.com/wp-content/up...
2,Batch Spicy Coconut Lime Margarita,{'minutes': '15'},-,{'minutes': '15'},"[jalapeños or serrano chiles, fresh basil leav...","{'jalapeños or serrano chiles': '2', 'fresh ba...",[1. Place the jalapeños (chiles) over a gas fl...,372 kcal,4,https://www.halfbakedharvest.com/wp-content/up...
3,Uncle Eddy’s White Russian,{'minutes': '10'},{'minutes': '5'},{'minutes': '15'},"[honey, cinnamon, nutmeg, vodka, bourbon or ru...","{'honey': '1/4', 'cinnamon': '2', 'nutmeg': '1...","[1. To make the cinnamon syrup. In a pot, comb...",373 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
4,Naughty Gingerbread Cocktail (+mocktail).,{'minutes': '5'},-,{'minutes': '7'},"[bourbon or spiced rum, orange or clementine j...","{'bourbon or spiced rum': '1 1/2', 'orange or ...","[1. If desired, rim your glass. On a small pla...",191 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
...,...,...,...,...,...,...,...,...,...,...
100,Winter Citrus Tequila Smash,{'minutes': '10'},-,{'minutes': '10'},"[orange zest, blood orange, peel removed and q...","{'orange zest': '1', 'blood orange, peel remov...","[1. In a cocktail shaker or glass jar, combine...",96 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
101,Christmas Snowstorm Margarita,{'minutes': '10'},-,{'minutes': '10'},"[unsweetened coconut milk, honey or granulated...","{'unsweetened coconut milk': '1 1/3', 'honey o...",[1. Bring the coconut milk and sugar to a simm...,252 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
102,Gingerbread White Russian,{'minutes': '5'},{'minutes': '5'},{'minutes': '10'},"[honey, molasses, fresh ginger, sliced, cinnam...","{'honey': '1/4', 'molasses': '3', 'fresh ginge...","[1. In a medium pot, combine 1 1/3 cups water,...",291 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
103,Harvest Apple Spritz.,{'minutes': '10'},{'minutes': '10'},{'minutes': '20'},"[honey, fresh ginger,, fresh sage leaves, plus...","{'honey': '1/4', 'fresh ginger,': '1', 'fresh ...","[1. In a small pot, bring 1/2 cup water, the h...",359 kcal,8,https://www.halfbakedharvest.com/wp-content/up...


### Recipes 25-200

In [15]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_2 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(150, 157):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_2['title'] = title
temp_recipes_2['prep_time'] = prep_time
temp_recipes_2['cook_time'] = cook_time
temp_recipes_2['total_time'] = total_time
temp_recipes_2['ingredients'] = ingredients
temp_recipes_2['ingredients with measurement'] = ingredients_measurment
temp_recipes_2['instructions'] = instructions
temp_recipes_2['calories'] = calories
temp_recipes_2['servings'] = servings
temp_recipes_2['images']= images



    

Proxies loaded: ['160.86.242.23:8080', '185.195.71.218:18080', '35.220.254.137:8080']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved 150 - https://www.halfbakedharvest.com/fig-bourbon-cider-smash/ with proxy 160.86.242.23:8080


<IPython.core.display.Javascript object>

...

Successfully retrieved 153 - https://www.halfbakedharvest.com/sweet-bourbon-peach-lemonade/ with proxy 160.86.242.23:8080
Proxy 185.195.71.218:18080 failed for https://www.halfbakedharvest.com/smashed-berry-rose-spritz/: HTTPSConnectionPool(host='www.halfbakedharvest.com', port=443): Max retries exceeded with url: /smashed-berry-rose-spritz/ (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 503 Service Unavailable')))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved 155 - https://www.halfbakedharvest.com/hibiscus-cherry-vodka-spritz/ with proxy 35.220.254.137:8080
ChunkedEncodingError occurred for proxy 160.86.242.23:8080 and https://www.halfbakedharvest.com/frozen-brunch-punch/: ('Connection broken: IncompleteRead(1885 bytes read, 8355 more expected)', IncompleteRead(1885 bytes read, 8355 more expected))


In [16]:
print(len(failed_url))

47


In [17]:
failed_url

['https://www.halfbakedharvest.com/frozen-watermelon-spritz/',
 'https://www.halfbakedharvest.com/hocus-pocus-old-fashioned/',
 'https://www.halfbakedharvest.com/pomegranate-punch/',
 'https://www.halfbakedharvest.com/the-20-most-popular-fall-drinks/',
 'https://www.halfbakedharvest.com/frosted-mistletoe-margarita/',
 'https://www.halfbakedharvest.com/spiced-cranberry-punch/',
 'https://www.halfbakedharvest.com/the-14-most-popular-thanksgiving-drinks/',
 'https://www.halfbakedharvest.com/voldemorts-dark-serum/',
 'https://www.halfbakedharvest.com/frostys-frosted-moscow-mule/',
 'https://www.halfbakedharvest.com/cranberry-thyme-spritz/',
 'https://www.halfbakedharvest.com/vampires-drip-cocktail/',
 'https://www.halfbakedharvest.com/the-deathly-hallows-cocktail/',
 'https://www.halfbakedharvest.com/fig-honey-thyme-prosecco-smash/',
 'https://www.halfbakedharvest.com/blood-orange-champagne-mule/',
 'https://www.halfbakedharvest.com/christmas-ale-sangria/',
 'https://www.halfbakedharvest.c

In [18]:
temp_recipes_2

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Fig Bourbon Cider Smash,{'minutes': '5'},-,{'minutes': '5'},"[apple cider, orange zest + 2 tablespoons juic...","{'apple cider': '1/4', 'orange zest + 2 tables...","[1. In a cocktail shaker, combine the apple ci...",52 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
1,Salted Watermelon Paloma,{'minutes': '10'},-,{'minutes': '10'},"[cubed watermelon, kosher salt, Florida’s Natu...","{'cubed watermelon': '2', 'kosher salt': 'seas...",[1. Add the watermelon to a blender and pulse ...,68 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
2,Blackberry Thyme Margarita,{'minutes': '5'},-,{'minutes': '5'},"[fresh blackberries, small red plum, quartered...","{'fresh blackberries': '8', 'small red plum, q...",[1. Run a lime wedge around the rim of your gl...,84 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
3,Sweet Bourbon Peach Lemonade,{'minutes': '5'},-,{'minutes': '5'},[Florida's Natural® Brand Lemonade with Raspbe...,{'Florida's Natural® Brand Lemonade with Raspb...,"[1. In a blender, combine the lemonade and pea...",136 kcal,8,https://www.halfbakedharvest.com/wp-content/up...
4,Hibiscus Cherry Vodka Spritz,{'minutes': '15'},{'minutes': '5'},{'minutes': '20'},"[fresh cherries, dried hibiscus flowers, honey...","{'fresh cherries': '3/4', 'dried hibiscus flow...",[1. Add the cherries to a medium size pot. Use...,158 kcal,4,https://www.halfbakedharvest.com/wp-content/up...


In [19]:
final = pd.concat([final, temp_recipes_2], ignore_index=True)
final

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images
0,Peach Thyme Smash,{'minutes': '5'},{'minutes': '5'},{'minutes': '10'},"[fresh peach slices, fresh thyme, peach jam/pr...","{'fresh peach slices': '1/2', 'fresh thyme': '...",[1. Optional: heat a grill or grill pan to hig...,226 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
1,-,-,-,-,-,-,-,-,-,https://www.halfbakedharvest.com/wp-content/up...
2,Batch Spicy Coconut Lime Margarita,{'minutes': '15'},-,{'minutes': '15'},"[jalapeños or serrano chiles, fresh basil leav...","{'jalapeños or serrano chiles': '2', 'fresh ba...",[1. Place the jalapeños (chiles) over a gas fl...,372 kcal,4,https://www.halfbakedharvest.com/wp-content/up...
3,Uncle Eddy’s White Russian,{'minutes': '10'},{'minutes': '5'},{'minutes': '15'},"[honey, cinnamon, nutmeg, vodka, bourbon or ru...","{'honey': '1/4', 'cinnamon': '2', 'nutmeg': '1...","[1. To make the cinnamon syrup. In a pot, comb...",373 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
4,Naughty Gingerbread Cocktail (+mocktail).,{'minutes': '5'},-,{'minutes': '7'},"[bourbon or spiced rum, orange or clementine j...","{'bourbon or spiced rum': '1 1/2', 'orange or ...","[1. If desired, rim your glass. On a small pla...",191 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
...,...,...,...,...,...,...,...,...,...,...
105,Fig Bourbon Cider Smash,{'minutes': '5'},-,{'minutes': '5'},"[apple cider, orange zest + 2 tablespoons juic...","{'apple cider': '1/4', 'orange zest + 2 tables...","[1. In a cocktail shaker, combine the apple ci...",52 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
106,Salted Watermelon Paloma,{'minutes': '10'},-,{'minutes': '10'},"[cubed watermelon, kosher salt, Florida’s Natu...","{'cubed watermelon': '2', 'kosher salt': 'seas...",[1. Add the watermelon to a blender and pulse ...,68 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
107,Blackberry Thyme Margarita,{'minutes': '5'},-,{'minutes': '5'},"[fresh blackberries, small red plum, quartered...","{'fresh blackberries': '8', 'small red plum, q...",[1. Run a lime wedge around the rim of your gl...,84 kcal,1,https://www.halfbakedharvest.com/wp-content/up...
108,Sweet Bourbon Peach Lemonade,{'minutes': '5'},-,{'minutes': '5'},[Florida's Natural® Brand Lemonade with Raspbe...,{'Florida's Natural® Brand Lemonade with Raspb...,"[1. In a blender, combine the lemonade and pea...",136 kcal,8,https://www.halfbakedharvest.com/wp-content/up...


In [20]:
def save_list_to_txt(filename, data_list):
    """
    Save a list of strings to a text file, one item per line.
    
    :param filename: The name of the file to save the list to.
    :param data_list: The list of strings to save.
    """
    with open(filename, 'w') as f:
        for item in data_list:
            f.write(f"{item}\n")

In [21]:
save_list_to_txt('failed_urls.txt', failed_url)
final.to_csv('drinks_1_end.csv', index=False)

### Recipes 201-500

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_3 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(201, 501):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_3['title'] = title
temp_recipes_3['prep_time'] = prep_time
temp_recipes_3['cook_time'] = cook_time
temp_recipes_3['total_time'] = total_time
temp_recipes_3['ingredients'] = ingredients
temp_recipes_3['ingredients with measurement'] = ingredients_measurment
temp_recipes_3['instructions'] = instructions
temp_recipes_3['calories'] = calories
temp_recipes_3['servings'] = servings
temp_recipes_3['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_3

In [ ]:
final = pd.concat([final, temp_recipes_3], ignore_index=True)
final

In [ ]:
def save_list_to_txt(filename, data_list):
    """
    Save a list of strings to a text file, one item per line.
    
    :param filename: The name of the file to save the list to.
    :param data_list: The list of strings to save.
    """
    with open(filename, 'w') as f:
        for item in data_list:
            f.write(f"{item}\n")

In [ ]:
save_list_to_txt('failed_urls_till501.txt', failed_url)
final.to_csv('desserts_201_501.csv', index=False)

### Recipes 501-800

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_4 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(501, 792):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = all_recipes_url[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_4['title'] = title
temp_recipes_4['prep_time'] = prep_time
temp_recipes_4['cook_time'] = cook_time
temp_recipes_4['total_time'] = total_time
temp_recipes_4['ingredients'] = ingredients
temp_recipes_4['ingredients with measurement'] = ingredients_measurment
temp_recipes_4['instructions'] = instructions
temp_recipes_4['calories'] = calories
temp_recipes_4['servings'] = servings
temp_recipes_4['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_4

In [ ]:
final = pd.concat([final, temp_recipes_4], ignore_index=True)
final

In [ ]:
save_list_to_txt('failed_urls_till201_792.txt', failed_url)
final.to_csv('recipes_501_792.csv', index=False)

In [ ]:
urls2 = []

with open('all_urls.txt', 'r') as file:
    for line in file:
        urls2.append(line.strip())

print(urls2)

In [ ]:
failed_url = []

### Recipes 801-1001

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_5 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(801, 1001):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = urls2[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_5['title'] = title
temp_recipes_5['prep_time'] = prep_time
temp_recipes_5['cook_time'] = cook_time
temp_recipes_5['total_time'] = total_time
temp_recipes_5['ingredients'] = ingredients
temp_recipes_5['ingredients with measurement'] = ingredients_measurment
temp_recipes_5['instructions'] = instructions
temp_recipes_5['calories'] = calories
temp_recipes_5['servings'] = servings
temp_recipes_5['images']= images



    

In [ ]:
final3 = pd.DataFrame(columns=columns)
final3

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_5

In [ ]:
final3 = pd.concat([final3, temp_recipes_5], ignore_index=True)
final3

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_6 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(1001, 1401):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = urls2[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_6['title'] = title
temp_recipes_6['prep_time'] = prep_time
temp_recipes_6['cook_time'] = cook_time
temp_recipes_6['total_time'] = total_time
temp_recipes_6['ingredients'] = ingredients
temp_recipes_6['ingredients with measurement'] = ingredients_measurment
temp_recipes_6['instructions'] = instructions
temp_recipes_6['calories'] = calories
temp_recipes_6['servings'] = servings
temp_recipes_6['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_6

In [ ]:
final3 = pd.concat([final3, temp_recipes_6], ignore_index=True)
final3

In [ ]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes_7 = pd.DataFrame(columns=columns)
title = []
prep_time = []
cook_time = []
total_time = []

ingredients_measurment = []
ingredients = []
instructions = []

calories = []
servings = []
images = []

# Loop over the desired number of pages
for i in range(1401, 1607):
    proxy = next(proxy_cycle)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = urls2[i]
    proxy_dict = {'http': proxy, 'https': proxy}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            extract(soup)
            print(f"Successfully retrieved {i} - {url} with proxy {proxy}")
        else:
            failed_url.append(url)
            print(f"Failed to retrieve page {url} with proxy {proxy}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        failed_url.append(url)
        print(f"Proxy {proxy} failed for {url}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        failed_url.append(url)
        print(f"ChunkedEncodingError occurred for proxy {proxy} and {url}: {e}")
        continue  # Move to the next page



temp_recipes_7['title'] = title
temp_recipes_7['prep_time'] = prep_time
temp_recipes_7['cook_time'] = cook_time
temp_recipes_7['total_time'] = total_time
temp_recipes_7['ingredients'] = ingredients
temp_recipes_7['ingredients with measurement'] = ingredients_measurment
temp_recipes_7['instructions'] = instructions
temp_recipes_7['calories'] = calories
temp_recipes_7['servings'] = servings
temp_recipes_7['images']= images



    

In [ ]:
len(failed_url)

In [ ]:
temp_recipes_7